# 🎨 AI Travel Content Generator with Amazon Nova

This notebook demonstrates automated travel content generation using [Amazon Nova Canvas](https://aws.amazon.com/bedrock/nova/) and [Amazon Nova Reel](https://aws.amazon.com/bedrock/nova/) through [Amazon Bedrock](https://aws.amazon.com/bedrock/). You'll create complete travel content packages including images, videos, and itineraries.

## What You'll Learn

- Generate destination images with Amazon Nova Canvas
- Create promotional videos with Amazon Nova Reel
- Build detailed travel itineraries with AI
- Automate content creation workflows
- Prepare multimodal content for AI agent processing

## Prerequisites

- AWS account with [Amazon Bedrock access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
- Amazon Nova Canvas and Nova Reel models enabled in Bedrock
- [Amazon S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-bucket.html) for content storage
- AWS CLI configured with appropriate permissions

## Content Generation Features
- ✅ Generate destination images with Amazon Nova Canvas
- ✅ Generate promotional videos with Amazon Nova Reels (saved locally)
- ✅ Create detailed travel itineraries
- ✅ Fully automated - no manual confirmations
- ✅ Ready-to-use content for multimodal AI agents

## 1. Setup

In [ ]:
import os
import boto3
from strands import Agent
from strands.models import BedrockModel
from strands_tools import generate_image, file_write, nova_reels, use_aws

# Disable tool consent prompts for automation
os.environ['BYPASS_TOOL_CONSENT'] = 'true'

print("✅ Setup complete")

## 2. Configuration

In [ ]:
# AWS Configuration
AWS_REGION = 'us-east-1'
S3_BUCKET = "your-bucket"  # Set your S3 bucket name for video generation or None

# Destination Configuration
DESTINATION = "Barcelona, Spain"
LANDMARK = "Sagrada Familia"
TRIP_DURATION = 5
TRAVEL_STYLE = "architectural"  # cultural, romantic, adventure, modern, historic, luxury
SEASON = "summer"  # spring, summer, fall, winter
GENERATE_VIDEO = True  # Set to False to skip video generation

print(f"🌍 Destination: {DESTINATION}")
print(f"🏛️  Landmark: {LANDMARK}")
print(f"📅 Duration: {TRIP_DURATION} days")
print(f"🎨 Style: {TRAVEL_STYLE}")
print(f"🌤️  Season: {SEASON}")
print(f"🎬 Generate Video: {'Yes' if GENERATE_VIDEO else 'No'}")
if GENERATE_VIDEO:
    print(f"📦 S3 Bucket: {S3_BUCKET}")

## 3. Create Content Generator Agent

In [ ]:
# Create Bedrock model
session = boto3.Session(region_name=AWS_REGION)
model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# Create content generator agent
content_agent = Agent(
    model=model,
    tools=[
        generate_image,
        nova_reels,
        use_aws,
        file_write
    ],
    system_prompt="""You are a professional travel content creator that generates complete content packages.

Your tools:
- generate_image: Create high-quality destination images with Amazon Nova Canvas (saves PNG locally)
- nova_reels: Generate promotional videos with Amazon Nova Reel (saves to S3)
- use_aws: Download files from S3 or perform other AWS operations
- file_write: Create text files for itineraries

When the user provides a destination, you MUST:

1. **Generate Destination Image**:
   - Use generate_image tool
   - Create a stunning photo of the main landmark at golden hour
   - Include detailed prompt with: lighting, composition, atmosphere, mood
   - Professional travel photography quality
   - The tool will save the PNG file automatically

2. **Generate Tour Video** (if S3 bucket provided):
   - First, use nova_reels tool with action="create"
   - Create a 6-second cinematic tour
   - CRITICAL: Text prompt MUST be under 512 characters
   - Show 3-4 iconic locations with smooth camera movement
   - Pass text, s3_bucket parameters
   - Save the invocation_arn from the response
   - Then, check status with nova_reels action="status" using the invocation_arn
   - When status is "Completed", use use_aws to download the video:
     * service_name: "s3"
     * operation_name: "download_file"
     * parameters: {"Bucket": bucket_name, "Key": video_key, "Filename": "output/[destination]-tour.mp4"}

3. **Create Travel Itinerary**:
   - Use file_write tool
   - Generate detailed day-by-day itinerary as text
   - Include: activities, restaurants, cultural insights, transportation tips
   - Match the travel style and season provided
   - Save to output/[destination]-itinerary.txt
   - Format with clear sections: Day 1, Day 2, etc.

Guidelines:
- Be creative and professional
- Use specific location names and landmarks
- Include atmospheric details
- Ensure video prompts are concise (under 512 chars)
- For video generation, wait for completion before downloading
- Generate ALL pieces of content automatically

Always execute all available tools when given a destination.
"""
)

print("🤖 Content Generator Agent created")

## 4. Generate Complete Content Package

This will automatically generate:
1. A destination image (landmark photo)
2. A promotional video
3. A detailed travel itinerary

In [ ]:
# Extract city and country from destination
city = DESTINATION.split(',')[0].strip()
country = DESTINATION.split(',')[1].strip() if ',' in DESTINATION else ""

# Check if video generation is possible
skip_video = not GENERATE_VIDEO or S3_BUCKET == "your-bucket-name" or not S3_BUCKET
video_instruction = "Skip video generation (disabled or no S3 bucket configured)." if skip_video else f"Generate video using S3 bucket: {S3_BUCKET}"

# Generate complete content package
response = content_agent(
    f"""
Generate a complete travel content package for:

Destination: {DESTINATION}
City: {city}
Country: {country}
Main Landmark: {LANDMARK}
Trip Duration: {TRIP_DURATION} days
Travel Style: {TRAVEL_STYLE}
Season: {SEASON}

Video generation: {video_instruction}
{'S3 Bucket: ' + S3_BUCKET if not skip_video else ''}

Create {'image and itinerary' if skip_video else 'all three pieces of content (image, video, and itinerary)'}.
Use the exact parameters provided above.

For video generation:
1. Use nova_reels to create the video (it will be saved to S3)
2. Check the status until completed
3. Use use_aws to download the video from S3 to data-sample/
"""
)

print("\n" + "="*80)
print("🎨 Content Generation Complete")
print("="*80)
print(response)

In [ ]:
response = content_agent("check and download de video")

## 5. Verify Generated Content

In [ ]:
from pathlib import Path

data_dir = Path("output")

if data_dir.exists():
    print("📁 Generated Content:\n")
    
    # Images
    images = list(data_dir.glob("*.jpg")) + list(data_dir.glob("*.png"))
    if images:
        print("🖼️  Images:")
        for img in images:
            size_mb = img.stat().st_size / (1024 * 1024)
            print(f"  • {img.name} ({size_mb:.2f} MB)")
    
    print()
    
    # Videos
    videos = list(data_dir.glob("*.mp4"))
    if videos:
        print("🎬 Videos:")
        for vid in videos:
            size_mb = vid.stat().st_size / (1024 * 1024)
            print(f"  • {vid.name} ({size_mb:.2f} MB)")
    
    print(f"\n✅ Total files: {len(list(data_dir.glob('*')))}")
else:
    print("❌ No content directory found")

## Summary

**What it does:** Generates complete travel content packages (images, videos, itineraries) using Strands Agents native tools.

**Tools:** `generate_image` (Nova Canvas), `nova_reels` (Nova Reel), `use_aws` (S3 operations), `file_write` (text files)

**Output location:** All content saved to `output/` directory

**Video workflow:** Generate in S3 → Check status → Download to `output/` (requires S3 bucket, takes 5-10 min)

**Requirements:** AWS credentials, optional S3 bucket for video generation

Set `GENERATE_VIDEO = False` to skip video generation if no S3 bucket available.